# **Homework 3 - Michelin restaurants in Italy**

*Group#20*

- **Marco Zimmatore** - [zimmatore.1947442@studenti.uniroma1.it](mailto:zimmatore.1947442@studenti.uniroma1.it)
- **Gabriele Cabibbo** - [cabibbo.2196717@studenti.uniroma1.it](mailto:cabibbo.2196717@studenti.uniroma1.it)
- **Emre Yeşil** - [1emreyesil@gmail.com](mailto:1emreyesil@gmail.com)
- **Emanuele Iaccarino** - [emanueleiaccarino.ei@gmail.com](mailto:emanueleiaccarino.ei@gmail.com)

___

In [40]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
import string
from nltk.stem import PorterStemmer
import contractions
from nltk.stem import WordNetLemmatizer
import re
import json
from collections import defaultdict
from DataCollection import engine

## **1. Data collection**

Although it would be ideal to call all functions together under ``` engine.py ``` and perform the parsing in a single step, this approach was not possible with our "more efficient" implementation. 

We’ll explore these issues later on, along with the solution implemented to address them.

Anyway I left ``` engine.py ``` file under DataColletion folder: the file works there but doesn't here on this Jupyter Notebook

### **1.1 Get the list of Michelin restaurants**

Fun Fact: I first did this part of the code 04/11/2024, the next day the number of restaurants was reduced from 2037 to 1983 (this is confirmed by the official site where now we have 100 pages instead of 102)

In [4]:
from DataCollection.crawler import get_michelin_urls
import logging
import time

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

start_time = time.time()
get_michelin_urls()
logging.info(f"Time to collect urls: {time.time() - start_time} seconds") # <3

DEBUG:root:Analyzing page 1
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): guide.michelin.com:443
DEBUG:urllib3.connectionpool:https://guide.michelin.com:443 "GET /en/it/restaurants/page/1 HTTP/1.1" 200 None
DEBUG:root:Analyzing page 2
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): guide.michelin.com:443
DEBUG:urllib3.connectionpool:https://guide.michelin.com:443 "GET /en/it/restaurants/page/2 HTTP/1.1" 200 None
DEBUG:root:Analyzing page 3
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): guide.michelin.com:443
DEBUG:urllib3.connectionpool:https://guide.michelin.com:443 "GET /en/it/restaurants/page/3 HTTP/1.1" 200 None
DEBUG:root:Analyzing page 4
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): guide.michelin.com:443
DEBUG:urllib3.connectionpool:https://guide.michelin.com:443 "GET /en/it/restaurants/page/4 HTTP/1.1" 200 None
DEBUG:root:Analyzing page 5
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): guide.mi

### **1.2 Crawl Michelin restaurant pages**

Using asyncio in the web scraping code allows to handle multiple HTML downloads concurrently, making the process faster and more efficient. Instead of waiting for each download to complete before starting the next (sequentially), the event loop jumps to the next available task when one is waiting for data, allowing to download multiple pages at the same time.

More info can be found [here](https://www.zenrows.com/blog/asynchronous-web-scraping-python#what-is-asynchronous-web-scraping), while the official Github repository is avaible at this [link](https://github.com/oxylabs/asynchronous-web-scraping-python)


 Apparently the reason why ``` engine.py ``` was working but calling here was not, is that jupyter run an event loop in the background that is not compatible with our asyncio library

In [9]:
from DataCollection.crawler import download_html_async

# I was having the following error: asyncio.run() cannot be called from a running event loop 
# solution was found here: https://community.openai.com/t/error-when-using-langchain-webresearchretriever-runtimeerror-asyncio-run-cannot-be-called-from-a-running-event-loop/341969/5
# and here: https://github.com/googlecolab/colabtools/issues/3720
async def main(): # This allow us to run the function asynchronously
    await download_html_async() 

await main()

# Apparently the reason why engine.py was working but calling here was not, is that jupyter run an event loop in the background that is not compatible with our asyncio library

INFO:root:Downloading batch 1
DEBUG:root:Saved HTML to page_1\restaurant_15.html
DEBUG:root:Saved HTML to page_1\restaurant_17.html
DEBUG:root:Saved HTML to page_1\restaurant_7.html
DEBUG:root:Saved HTML to page_1\restaurant_9.html
DEBUG:root:Saved HTML to page_1\restaurant_6.html
DEBUG:root:Saved HTML to page_1\restaurant_5.html
DEBUG:root:Saved HTML to page_1\restaurant_10.html
DEBUG:root:Saved HTML to page_1\restaurant_1.html
DEBUG:root:Saved HTML to page_1\restaurant_18.html
DEBUG:root:Saved HTML to page_1\restaurant_13.html
DEBUG:root:Saved HTML to page_1\restaurant_8.html
DEBUG:root:Saved HTML to page_1\restaurant_12.html
DEBUG:root:Saved HTML to page_1\restaurant_4.html
DEBUG:root:Saved HTML to page_1\restaurant_19.html
DEBUG:root:Saved HTML to page_1\restaurant_3.html
DEBUG:root:Saved HTML to page_1\restaurant_14.html
DEBUG:root:Saved HTML to page_1\restaurant_11.html
DEBUG:root:Saved HTML to page_1\restaurant_20.html
DEBUG:root:Saved HTML to page_1\restaurant_16.html
DEBUG:roo

CancelledError: 

In [6]:
# moves folders under same main folder for cleaness
from DataCollection.organize_folders import organize_folders
organize_folders()

Created main folder: michelin_restaurants
All page_* folders have been organized into the main folder.


### **1.3 Parse downloaded pages**

With the same method, we use asyncio for parsing, allowing to process multiple HTML files at the same time.
Additionally, the use of batches allow us to control the parallelism without overloading our sources, this way we analyze 100 files per batch. 

Smart and efficient way to perform it!

In [7]:
from DataCollection.parser import parse_all_restaurants
import asyncio
await parse_all_restaurants()

INFO:root:Starting to parse all restaurants.
DEBUG:root:TSV file header written.
INFO:root:Processing batch 1 with 100 files.
INFO:root:Processing batch 2 with 100 files.
INFO:root:Processing batch 3 with 100 files.
INFO:root:Processing batch 4 with 100 files.
INFO:root:Processing batch 5 with 100 files.
INFO:root:Processing batch 6 with 100 files.
INFO:root:Processing batch 7 with 100 files.
INFO:root:Processing batch 8 with 100 files.
INFO:root:Processing batch 9 with 100 files.
INFO:root:Processing batch 10 with 100 files.
INFO:root:Processing batch 11 with 100 files.
INFO:root:Processing batch 12 with 100 files.
INFO:root:Processing batch 13 with 100 files.
INFO:root:Processing batch 14 with 100 files.
INFO:root:Processing batch 15 with 100 files.
INFO:root:Processing batch 16 with 100 files.
INFO:root:Processing batch 17 with 100 files.
INFO:root:Processing batch 18 with 100 files.
INFO:root:Processing batch 19 with 100 files.
INFO:root:Processing batch 20 with 82 files.
INFO:root

In [10]:
import shutil
folder_path = "michelin_restaurants"
shutil.rmtree(folder_path)
print(f"The folder '{folder_path}' has been deleted successfully.")
# it's too heavy to push it and anyway we don't neeed it anymore 

FileNotFoundError: [WinError 3] Impossibile trovare il percorso specificato: 'michelin_restaurants'

In [11]:
from DataCollection.file_type_converter import tsv_to_csv
tsv_to_csv('michelin_restaurants_data.tsv', 'michelin_restaurants_data.csv')

Conversion complete: michelin_restaurants_data.csv


## **2. Search Engine**

___

### **2.0 Preprocessing**

#### **2.0.1 Preprocessing the Text**

In [38]:
# Load The Datframe
df = pd.read_csv("michelin_restaurants_data.csv")

In [41]:
# Set up the stopwords list
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
def remove_stopwords(text):
    if isinstance(text, str):


        words=text.split()
        words=[word for word in words if word.lower() not in stop_words]
        cleaned_text = ' '.join(words)

        return cleaned_text
    else:
            return ""  



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\emanu\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [42]:
df['no_stopwords']=df['description'].apply(remove_stopwords)

In [43]:

# Set options to display all columns, rows, and no truncation
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)

In [44]:
print(df[['description', 'no_stopwords']].head())


                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             description  \
0                                                                                                                           

In [45]:
def remove_punc(text):
    if isinstance (text,str):
         return text.translate(str.maketrans('', '', string.punctuation))
    else:
        return ""       

In [46]:
df['no_stopwords_and_punct'] = df['no_stopwords'].apply(remove_punc)

# Display the first few rows to verify the result
print(df[['description', 'no_stopwords', 'no_stopwords_and_punct']].head())

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             description  \
0                                                                                                                           

In [47]:
stemmer=PorterStemmer()

def apply_stemming(text):
    words= text.split()
    stemmed_words=[stemmer.stem(word) for word in words]
    stemmed_text=' '.join(stemmed_words)
    return stemmed_text

In [48]:
# Apply stemming to the 'no_stopwords_and_punct' column
df['no_stpwrd_punc_and_stemmed'] = df['no_stopwords_and_punct'].apply(apply_stemming)

# Display the first few rows to verify the result
print(df[['description', 'no_stopwords_and_punct', 'no_stpwrd_punc_and_stemmed']].head())

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             description  \
0                                                                                                                           

In [49]:
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()

def apply_lemmatization(text):
    text = text.lower()
    words = text.split()
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
    lemmatized_text = ' '.join(lemmatized_words)  
    return lemmatized_text

df['lemmatized_text'] = df['no_stopwords_and_punct'].apply(apply_lemmatization)
print(df[['description', 'no_stopwords', 'no_stopwords_and_punct', 'lemmatized_text']].head())


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\emanu\AppData\Roaming\nltk_data...


                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             description  \
0                                                                                                                           

After I checked the results of Stemming  and lemmatizer, I prefer to use lemmatized's output. Because I realized that,Stemming caused same meaningless words. I think this method uses directly throw away same letters from end of the words. As a result, according to outputs of two methods I will contuinue with lemmatizer. (For example:restaurant>>>restaur)


In [50]:
# Removing Extra Whitespace

df['lemmatized_text+'] = df['lemmatized_text'].str.replace(r'\s+', ' ', regex=True).str.strip()


# Removing  Non-ASCII Characters or Emojis
df['cleaned_text'] = df['lemmatized_text+'].str.encode('ascii', 'ignore').str.decode('ascii')


In [51]:
#Expanding Contractions
df['cleaned_text'] = df['cleaned_text'].apply(lambda x: contractions.fix(x))


In [52]:
#Removing URLs and Email Addresses
df['cleaned_text'] = df['cleaned_text'].str.replace(r'http\S+|www\S+|https\S+', '', regex=True)
df['cleaned_text'] = df['cleaned_text'].str.replace(r'\S+@\S+', '', regex=True)
df['restaurant_name'] = df['restaurant_name'].str.split('–|-').str[0].str.strip()
df['restaurant_name'].head(5)

0                        O Me O Il Mare
1                        Alessandro Feo
2                                  Dama
3    Il Ristorante Alain Ducasse Napoli
4                         Palazzo Utini
Name: restaurant_name, dtype: object

### **2.1 Conjunctive Query** 

#### **2.1.1 Create Your Index!**

In [53]:
# Create a unique document ID for each restaurant entry in the DataFrame
df['document_id'] = df.index  


vocab = {}
current_term_id = 0
inverted_index = defaultdict(list)  

# Function to get or assign a term ID
def get_term_id(word):
    """Assigns a unique ID to each word in the vocabulary."""
    global current_term_id
    # If the word is not already in the vocabulary, add it with a new ID
    if word not in vocab:
        vocab[word] = current_term_id
        current_term_id += 1  
    return vocab[word]  

# Go through each restaurant's description and build the vocabulary and index
for _, row in df.iterrows():
    doc_id = row['document_id']
    words = row['cleaned_text'].split()  
    unique_words = set(words)  

    # For each unique word, get its term ID and add the document ID to the inverted index
    for word in unique_words:
        term_id = get_term_id(word)
        inverted_index[term_id].append(doc_id)

# Convert the vocabulary dictionary to a DataFrame and save it to a CSV file for future reference
vocab_df = pd.DataFrame(list(vocab.items()), columns=['word', 'term_id'])
vocab_df.to_csv('vocabulary.csv', index=False)

# Save the inverted index as a JSON file to use later when processing queries
with open('inverted_index.json', 'w') as f:
    json.dump(inverted_index, f)

# Load the inverted index and vocabulary when we need to process a search query
with open('inverted_index.json', 'r') as f:
    inverted_index = json.load(f)

# Reload vocabulary from CSV into a dictionary
vocab_df = pd.read_csv('vocabulary.csv')
vocab = dict(zip(vocab_df['word'], vocab_df['term_id']))

#### **2.1.2 Execute the Query**

In [54]:

# Save inverted index to JSON
with open('inverted_index.json', 'w') as f:
    json.dump(inverted_index, f)

# Load inverted index and vocabulary (for query processing)
with open('inverted_index.json', 'r') as f:
    inverted_index = json.load(f)

vocab_df = pd.read_csv('vocabulary.csv')
vocab = dict(zip(vocab_df['word'], vocab_df['term_id']))

# Query preprocessing function
def preprocess_query(query):
    """Processes a user query to remove punctuation and stop words, returning term IDs."""
    query = query.lower()
    query = re.sub(r'[^\w\s]', '', query)  # Remove punctuation
    query_terms = [word for word in query.split() if word not in stop_words] 
    # Convert each word to its term ID if it's in the vocabulary
    term_ids = [vocab.get(word) for word in query_terms if word in vocab]
    return term_ids  # Return the list of term IDs for the query

# %% Search Function
def search_restaurants(query):
    """Finds and returns restaurants that match all words in the query."""
    term_ids = preprocess_query(query)
    if not term_ids:
        print("No matching terms found in the vocabulary.")
        return pd.DataFrame()  # No matching terms

    # Get document IDs that contain all the query terms
    matching_docs = set(inverted_index.get(str(term_ids[0]), []))  
    for term_id in term_ids[1:]:
        matching_docs.intersection_update(inverted_index.get(str(term_id), [])) 

    # If there are matches, retrieve their restaurant details
    if matching_docs:
        results = df[df['document_id'].isin(matching_docs)][['restaurant_name', 'address', 'description', 'url']]
        return results.head(5)
    else:
        print("No documents match all query terms.")
        return pd.DataFrame()  # No matching documents



In [55]:
# Example usage
query = "moder cuisine restaurant"
results = search_restaurants(query)
print(results)

                       restaurant_name                    address  \
0                       O Me O Il Mare             Via Roma 45/47   
3   Il Ristorante Alain Ducasse Napoli  Via Cristoforo Colombo 45   
7        La Trattoria Enrico Bartolini           Località Badiola   
10                              Ménage        Via Euplio Reina 13   
11                          Donevandro            via Garibaldi 2   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

### **2.2 Ranked Search Engine with TF-IDF and Cosine Similarity**

#### **2.2.1 Inverted Index with TF-IDF Scores**

In [160]:
import importlib
importlib.reload(engine)


<module 'DataCollection.engine' from 'c:\\Users\\Marco\\Desktop\\Data Science Magistrale\\ADM\\ADM-HW3\\ADM-HW3\\DataCollection\\engine.py'>

1. **tfIdf Scores**: Calculate TF-IDF scores for each term in each restaurant’s description.


In [162]:
df_tfidf = df.copy()
vocabulary = pd.read_csv("vocabulary.csv")

with open("inverted_index.json", 'r', encoding='utf-8') as file:
    inverted_index = json.load(file)
updated_inverted_index = engine.update_inverted_index(df_tfidf, inverted_index, vocabulary)

2. **Updated Inverted Index**: Build a new inverted index where each entry is a term, and the value is a list of tuples containing document IDs and TF-IDF scores.

In [80]:
# Finally we load the inverted_index in a JSON file
with open("updated_inverted_index.json", "w") as file:
    json.dump(updated_inverted_index, file)


#### **2.2.2 Execute the Ranked Query**


1. Process the query terms.

In [163]:
query = "MODERN; Seasonal CUISINE."

query_series = pd.Series(query)
query_series=query_series.apply(remove_stopwords).apply(apply_stemming).apply(remove_punc).apply(apply_lemmatization)
query_preprocessed = query_series.iloc[0]
query_preprocessed

'modern season cuisine'

2. Use Cosine Similarity to rank matching restaurants based on the TF-IDF vectors of the query and each document.


In [164]:
similarities = engine.rank_documents(query_preprocessed, updated_inverted_index, df, vocabulary)

{3: [('162', 0.017414950922279588), ('8', 0.0067800391143437)], 6: [('162', 0.019938856853044747)], 9: [('162', 0.05981657055913424)], 11: [('162', 0.03355563714292897), ('8', 0.013063977805686643)], 19: [('162', 0.04438003622129315)], 22: [('162', 0.039308032081716786), ('8', 0.015303516858090066)], 28: [('162', 0.03355563714292897)], 36: [('162', 0.016378346700715328), ('933', 0.08000654137987041), ('8', 0.012752930715075056)], 37: [('162', 0.017638219523847275), ('8', 0.006866962692732722)], 40: [('162', 0.02990828527956712), ('8', 0.011643980218112008)], 42: [('162', 0.03126775279227472), ('8', 0.012173252046208009)], 43: [('162', 0.05095485640222546), ('8', 0.019837892223450084)], 45: [('162', 0.02372036418724289), ('933', 0.057935771344044096), ('8', 0.009234880862640557)], 55: [('162', 0.08598632017875547)], 63: [('162', 0.039308032081716786), ('8', 0.015303516858090066)], 64: [('162', 0.057324213452503645), ('8', 0.022317628751381345)], 65: [('162', 0.02927193878425718)], 70: [

3. Return the top-k results or all matching restaurants if fewer than k have non-zero similarity.

In [170]:
k = 5  
top_k_values = similarities[:k]
documents_best_k = df_tfidf.iloc[[top_k_index[0] for top_k_index in top_k_values]]
documents_best_k["similarity_score"] = [values[1] for values in top_k_values]
documents_best_k[["restaurant_name", "address", "description", "url", "similarity_score"]]

C:\Users\Marco\AppData\Local\Temp\ipykernel_38156\1052310851.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  documents_best_k["similarity_score"] = [values[1] for values in top_k_values]


,restaurant_name,address,description,url,similarity_score
1605,Sintesi,viale dei Castani 17,"A modern, welcoming restaurant whose motto “Tradition & Innovation” perfectly sums up its cuisine. On the à la carte and tasting menus alike you’ll find classic dishes including risotto with peas, lemon and raw scampi alongside more creative offerings such as grilled pannicolo (hanger steak) with Nemi strawberries and herbs, all presented in a modern and precise manner, and with the occasional nod to Asian techniques and northern European culinary methods (fermentation, marinades, dry aging of fish etc). The wine list changes with the seasons and the menu.",http://ristorantesintesi.it,0.866468
890,San Giorgio,viale Brigate Bisagno 69r,"Situated in the city albeit not right in the centre, San Giorgio is a typical Genovese restaurant – elegant and classic in style, with excellent cuisine served by the Scala family who have been in charge for decades. Young chef Samuele Di Mauro is at the helm, having started here as an apprentice and worked his way to the position of sous-chef and then head chef in 2022. Despite a focus on modern presentation and techniques, as demonstrated in dishes such as the raw bluefin tuna seasoned with an intense verbena mousse and served with caper cream and a “caviar” of dried and fried tuna, his cuisine is Mediterranean in style, with its roots in Ligurian aromas and flavours. The wine list is as impressive as the cuisine, with an entire section dedicated to French and Italian sparkling wines.",https://www.ristorantesangiorgiogenova.it/,0.825504
298,Sissi,via Galilei 44,"Situated just outside the pedestrianised area of beautiful Merano, the main dining room of this restaurant has a welcoming feel and is decorated in attractive Art-Nouveau style. Here, owner-chef Andrea Fenoglio divides his time between the kitchen and front of house so that he can take time to chat with guests. His cuisine has solid classic foundations, with a good choice of dishes on the à la carte, plus a surprise tasting menu (Settepiatti ) which changes every year (as does the ceramic tableware on which it is served!). The cuisine is comforting and full of generous flavours, to which Fenoglio adds a light modern touch with his carefully chosen accompaniments and presentation. Whatever dish you choose, you can be sure of finding the perfect wine pairing for your meal on the skilfully constructed wine list.",https://sissi.andreafenoglio.com/it/home/1-0.html,0.819180
1186,Vignamare,"strada Castello, 20 - Colla Micheri","From Andora, head uphill along a winding road, then leave the car in the car park and follow a path through the vineyards to get to this restaurant, where meals are served either on a panoramic terrace (in fine weather) or indoors in a modern, wood-furnished dining room. Wherever you choose, the regionally influenced cuisine (often with a focus on vegetables) is extraordinary, with many ingredients grown or produced on the property. There are three tasting menus of various lengths, all of which showcase Ligurian cuisine. Similarly, the wine is sourced exclusively from the restaurant’s own estate.",https://www.peqagri.it/pages/ristorante-vignamare,0.819180
1612,Pentole e Provette,Via Musco 37,"This restaurant run by Luca Trabalzini and his partner Rossella is a real gem in the small town of Fasano. Their story, which began in Australia, adds a fascinating touch to this gastronomic project. The atmosphere here is warm and lively, making this the perfect setting in which to enjoy the chef’s innovative cuisine, which he showcases on three distinct menus dedicated to meat, fish and vegetarian dishes respectively. Luca’s cuisine is modern and elegant, with particular attention paid to flavours and textures so that each and every dish offers a fully rounded sensorial experience.",http://www.pentoleeprovette.com,0.819180


## **3. Define a New Score!**

___

1. **User Query**: The user provides a text query. We’ll retrieve relevant documents using the search engine built in Step 2.1.

2. **New Ranking Metric**: After retrieving relevant documents, we’ll rank them using a new custom score. Instead of limiting the scoring to only the description field, we can include other attributes like `priceRange`, `facilitiesServices`, and `cuisineType`.

3. You will use a **heap data structure** (e.g., Python’s heapq library) to maintain the *top-k* restaurants.


## **4. Visualizing the Most Relevant Restaurants**

___

Geocode Locations were already parsed when scraping the HTML files, since the info was contained in the webpage and could be extracted using REGEX on the link that connected to Google Maps

In [12]:
!pip install folium


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## Why Folium

We chose **Folium** due to its simplicity, flexibility, and all the integration avaible.

### Key Benefits of Using Folium
- **Interactive Maps**: Allows users to zoom, pan, and explore different locations dynamically.
- **Customization**: Supports adding custom markers, color coding, and popups to display detailed information.
- **Plugin Integration**: Folium supports various plugins that enhance interactivity

For more details: **[Folium Documentation](https://python-visualization.github.io/folium/)**.

---

## Explanation of Folium Plugins Used


### 1. MarkerCluster

The MarkerCluster plugin groups nearby markers into clusters, which expand to show individual markers as the user zooms in. It prevents overcrowding of markers on the map on high density area.

**Documentation**: [MarkerCluster Documentation](https://python-visualization.github.io/folium/latest/user_guide/plugins/marker_cluster.html)

---

### 2. MiniMap

The MiniMap plugin adds a mini-map to the right-bottom corner of the main map, helping users orient themselves within the broader region.

**Documentation**: [MiniMap Documentation](https://python-visualization.github.io/folium/latest/user_guide/plugins/mini_map.html)

---

### 3. HeatMap

The HeatMap plugin creates a heat map overlay that highlights areas with a high concentration of restaurants.

**Documentation**: [HeatMap Documentation](https://python-visualization.github.io/folium/latest/user_guide/plugins/heatmap.html)

---

### 4. MeasureControl

The MeasureControl plugin adds a tool to measure distances directly on the map. 

**Documentation**: [MeasureControl Documentation](https://python-visualization.github.io/folium/latest/user_guide/plugins/measure_control.html)

---

### 5. LocateControl

The LocateControl plugin adds a button that centers the map on the user’s current location.

**Documentation**: [LocateControl Documentation](https://python-visualization.github.io/folium/latest/user_guide/plugins/locate_control.html)


In [ ]:
import folium
import pandas as pd
from folium.plugins import MarkerCluster, HeatMap, MiniMap, MeasureControl, LocateControl

def create_restaurant_map(csv_file, output_map="restaurants_map.html"):
    data = pd.read_csv(csv_file)
    
    data['latitude'] = pd.to_numeric(data['latitude'], errors='coerce')
    data['longitude'] = pd.to_numeric(data['longitude'], errors='coerce')
    
    data = data.dropna(subset=['latitude', 'longitude'])
    
    # to center the map
    avg_lat = data['latitude'].mean()
    avg_lon = data['longitude'].mean()
    
    # Initialize map 
    m = folium.Map(location=[avg_lat, avg_lon], zoom_start=6, control_scale=True)

    # plugins
    marker_cluster = MarkerCluster().add_to(m) #group nearby markers to make it easier to explore the map (specially in high density zones)
    minimap = MiniMap().add_to(m) # improves navigation
    HeatMap([[row['latitude'], row['longitude']] for _, row in data.iterrows()]).add_to(m) # highlights areas with high concentrations of restaurants
    m.add_child(MeasureControl()) # tool to calculate distances directly on the map
    LocateControl().add_to(m) # to retrieve users current position

    # color mapping for price ranges
    price_color_mapping = {
        "€": "green",
        "€€": "blue",
        "€€€": "orange",
        "€€€€": "red"
    }
    
    # pop-up HTML for each restaurant so we can provide all the needed info when u click on it
    def create_popup_html(row):
        info = f"""
        <strong>{row['restaurant_name']}</strong><br>
        <em>{row['cuisine_type']}</em><br>
        <br><strong>Address:</strong> {row['address']}, {row['city']}, {row['postal_code']}, {row['country']}<br>
        <strong>Price Range:</strong> {row['price_range']}<br>
        <strong>Phone:</strong> {row['phone_number']}<br>
        <strong>Website:</strong> <a href="{row['url']}" target="_blank">{row['url']}</a><br>
        <strong>Description:</strong> {row['description']}<br>
        <br><strong>Opening Hours:</strong><br>
        <table style="width:100%; border:1px solid black;">
            <tr><td><strong>Monday</strong></td><td>{row['monday_hours']}</td></tr>
            <tr><td><strong>Tuesday</strong></td><td>{row['tuesday_hours']}</td></tr>
            <tr><td><strong>Wednesday</strong></td><td>{row['wednesday_hours']}</td></tr>
            <tr><td><strong>Thursday</strong></td><td>{row['thursday_hours']}</td></tr>
            <tr><td><strong>Friday</strong></td><td>{row['friday_hours']}</td></tr>
            <tr><td><strong>Saturday</strong></td><td>{row['saturday_hours']}</td></tr>
            <tr><td><strong>Sunday</strong></td><td>{row['sunday_hours']}</td></tr>
        </table><br>
        <strong>Facilities:</strong> {row['facilities_services']}<br>
        <strong>Credit Cards:</strong> {row['credit_cards']}
        """
        return info
    
    # markers
    for _, row in data.iterrows():
        lat, lon = row['latitude'], row['longitude']
        price_range = row['price_range']
        
        marker_color = price_color_mapping.get(price_range, "gray")
        
        # pop up to store info for each restaurant
        popup_html = create_popup_html(row)
        popup = folium.Popup(popup_html, max_width=300)
        
        # Create a marker for each restaurant
        folium.Marker(
            location=[lat, lon],
            popup=popup,
            tooltip_text = f"{row['restaurant_name']} - {row['price_range']} - {row['cuisine_type']}",
            icon=folium.Icon(color=marker_color, icon="cutlery", prefix="fa") # to make it more pretty I added cutlery icon from FontAwesome
        ).add_to(marker_cluster)
    
    #legend for price range
    legend_html = '''
     <div style="position: fixed; 
                 bottom: 50px; left: 50px; width: 150px; height: 140px; 
                 background-color: white; border:2px solid grey; z-index:9999; font-size:14px;">
     <strong>Price Range Legend</strong><br>
     <i style="color:green; font-size:15px;">&#9679;</i> €<br>
     <i style="color:blue; font-size:15px;">&#9679;</i> €€<br>
     <i style="color:orange; font-size:15px;">&#9679;</i> €€€<br>
     <i style="color:red; font-size:15px;">&#9679;</i> €€€€<br>
     </div>
     '''
    m.get_root().html.add_child(folium.Element(legend_html))
    
    m.save(output_map)
    print(f"Map has been saved to {output_map}")

create_restaurant_map("michelin_restaurants_data.csv")


Map has been saved to restaurants_map.html


## **5. BONUS: Advanced Search Engine**

___

## **Algorithmic Question (AQ)**
___

1. Write the pseudocode for an algorithm that solves this problem.


### **Overview**
The algorithm is designed to guide a robot that collects packages located on a grid, starting from the origin point (0,0). The robot can only move right ('R') or up ('U'). The goal is to collect all packages using the fewest moves while maintaining a lexicographically smallest path.


#### **Input**
- The number of test cases, `t`.
- For each test case:
  - The number of packages, `n`.
  - The coordinates `(x_i, y_i)` of each package.

#### **Output**
- For each test case, print "YES" followed by the smallest path if all packages can be collected, or "NO" if it is impossible.


### **Pseudocode**
- The idea behind the followed *pseudocode* is that we want to divide each problem into smaller problems. Instead of thinking about "How can we collect all the packages in the best way?", we can firstly solve the problem "How can i reach in the fastes way the next package?" and iterate over the packages.
- To develop this in the most efficient way, we have to sort the collection that will contain the coordinates of each package we are receiving in input, because we dont want to solve in the brute force way trying every path possible. To do that
we compute the distance between the origin and each of the coordinates and we sort them on this distance. 
- Then we compute the **Manhattan Distance** (or **L1 distance**) that helps up to compute the number of single steps from point A to point B:
$$ d = |x_1 - x_2| + |y_1 - y_2| $$
- We iterate over this number and in each iteration we decide to go up or right giving more value to the right because lexicographycally  $"R" < "U"$
- Finally return the Path scaling this idea

Here the pseudocode:

```python
    FOR each test_case FROM 1 TO t DO
      n = READ integer from input
      list_of_coordinates = []

      FOR each package FROM 1 to N DO
        coordinates = READ integers from input
        ADD coordinates to list_of_coordinates

      sorted_coordinates = sort(list_of_coordinates)

      FOR target_x, target_yy IN sorted_coordinates DO
        if target_x or target_y LESS THAN current_coordinates
          BREAK
          PRINT("NO")

        COMPUTE DISTANCE BETWEEN target_x, target_y AND current_coordinates

        FOR steps FROM 1 TO distance DO
          IF current_coordinate_x LESS THAN target_x 
            THEN MOVE RIGHT AND ADD "R" TO PATH
            CONTINUE

          ELSE IF current_coordinate_Y LESS THAN target_y
            THEN MOVE UP AND ADD "U" TO PATH

    RETURN PATH


```




Then we have implemented in python this solution using numpy package to make the sorting more efficient:

In [ ]:
import numpy as np 
import functions

with open("algorithmic_input.txt") as file:

    # Select the number of Test Cases
    t = int(file.readline().strip())
    # Iterate over the number fo test cases
    for _ in range(t):
        
        # Extract the number of Packages to collect
        number_packages = int(file.readline().strip())

        # Store all the input coordinate into the Numpy Array
        list_of_coordinates = np.array([tuple(map(int, file.readline().strip().split())) for _ in range(number_packages)] , dtype = int)

        # I compute the distance from the (0,0) point to sort the list
        distances = np.sqrt(list_of_coordinates[:, 0]**2 + list_of_coordinates[:, 1]**2)

        # I have used the argsort function in order to return the indices of the distances ordered in ascendent way
        sorted_indices = np.argsort(distances)

        # I return the list ordered
        sorted_coordinates = list_of_coordinates[sorted_indices]

        # Execute the function to return the smallest path given an array of sorted coordinates
        functions.compute_smallest_path(sorted_coordinates)

    




YES
RUUURRRRUU
NO
YES
RRRRUUU


2. Prove that your algorithm is correct.

To prove the correctness of our algorithm, which contains a loop, the best way to do it is applying the **Invariant of the Loop**.
It is a logic condition that remains true at the beginning and end of each iteration of a loop during the execution of an algorithm.
Showing that a loop has an invariant can help to prove that the loop behaves correctly and in general the correctness of the algorithm.

#### **Invariant of the Loop**

**Loop Invariant**: At the beginning of each iteration, the current_coordinate represents a point reachable from the origin using only "U" and "R" moves, and result contains a valid path from the origin to current_coordinate.

**Proof**:

- **Initialization**: The starting point is (0, 0), which is trivially reachable from the origin.

- **Maintenance**:
The algorithm checks if the next coordinate (`target_x`, `target_y`) can be reached using only "U" and "R". This is verified by checking that `target_x` is not less than `current_coordinate[0]` and `target_y` is not less than `current_coordinate[1]`.
If this condition fails, the algorithm outputs "NO" and terminates, proving that reaching all points in the list is impossible with the allowed moves.
If the condition holds, the algorithm updates current_coordinate and appends "R" and "U" moves accordingly, ensuring that only valid moves are added to result.

- **Termination**: The loop iterates over all coordinates in list. When it completes without triggering the "NO" condition, it implies that all coordinates were reachable from the previous position using only "U" and "R" moves.

**Conclusion**: The loop invariant holds true throughout each iteration, guaranteeing that result represents a valid path up to `current_coordinate`.

Obviously we can scale up the invariant also for the external loop over the number_of_packages to complete the proof.

___
3. Compute the time complexity of your algorithm in Big O notation. Break down the steps involved in the algorithm, and explain which parts contribute most to the overall time complexity.
___

To Compute the time complexity of the algorithm int the **BIG O** Notation we have to consider in the *worst case*  the operation in your algorithm that consumes the most execution time.

The *worst case* happens when we are able to collect all the packages in input, so there is no stopping condition.

Let's break down the steps in the algorithm.
We define $t$ as the number of `test cases` and $n$ as the number of `packages` our robot has to collect each test case $t$.

- So at first we have to execute a for loop to iterate over the `test cases` t, so everything we'll be describe below will be executed $t$ times:
    ```python
                            for _ in range(t):
    ```
    - Then we use a list comprehension (*python functionality*) to store all the coordinates of the `packages` for that test case:
    ```python
    list_of_coordinates = np.array([tuple(map(int, file.readline().strip().split())) for _ in range(number_packages)] ,dtype = int)
    ```
    - The previous instruction maps a series of $n$ tuples (`number of packages`) inside the numpy array, so the instruction 
    ```python 
                            tuple(map(int, file.readline().strip().split())) 
    ``` 
    - will be executed $n$ times.

    - The **fundamental instruction** of the algorithm, is:
    ```python 
                            sorted_indices = np.argsort(distances)
    ```
    - Infact, as we know in the worst case the most efficient **Sorting algorithms** based on comparisons like *Merge Sort* have a lower bound of $O(n\log(n))$

Even if the previous instruction consumes the most execution time, let's be specific in the calculus of complexity and dive in the remaining part of the code.

- the function `compute_smallest_path` takes in input the *list_of_sorted_coordinates* and outputs the smallest path to collect all the packages if it is possible:


    - Ignoring instructions of initialization we consider $O(1)$ computationally, we iterate over the input list of size $n$ that contains the coordinates of each package sorted by the distance to the origin:

```python 
                            for target_x, target_y in list:
``` 

- The *fundamental operation* in this function is this loop, that in the worst case will be executed $s_i$ times where $s_i$ is the single step distance between the i-th and i+1-th package. 

```python 
                            for _ in range(distance_from_next):

``` 

- If we call $p$ the number of total steps, $p =  \sum_{i=1}^{n} s_i $


So the algorithm will have time complexity equal to $O(t \times n\log(n)) + O(t \times p \times n)$



___
4. Ask an LLM tool (such as ChatGPT, Claude AI, Gemini, Perplexity, etc.) to evaluate the time complexity of your code using Big O notation. Is the assessment accurate? If it differs from your previous analysis, which would be correct? Please explain your reasoning.
___

> My prompt to *ChatGpt* was `"##CODE## evaluate the time complexity of this code using Big O notation "`.

This was the answer:

To analyze the time complexity of this code, let’s break down each section and determine the number of operations relative to $n$, where $n$ represents the number of packages.

#### 1. **Reading and Parsing Input**
   - **File Reading**: The code reads $t$ test cases from a file. For each test case, it reads the number of packages and coordinates.
   - **Parsing Coordinates**: Each coordinate pair is read and converted into an integer tuple, which is stored in a Numpy array. Parsing takes $O(n)$ time per test case since it reads $n$ coordinates, each in $O(1)$ time.
   
   **Time Complexity for Parsing Input**: $O(t \times n)$.

#### 2. **Calculating Distances**
   - For each coordinate, the distance from the origin $0,0$ is calculated using the Euclidean formula $\sqrt{x^2 + y^2}$.
   - Calculating the distance for each of the $n$ points takes $O(n)$ time, as computing the distance for each point is $O(1)$.

   **Time Complexity for Distance Calculation**: $O(t \times n)$.

#### 3. **Sorting Coordinates by Distance**
   - The code sorts the indices based on their distances from the origin. Sorting an array of $n$ elements takes $O(n \log n)$.
   - After sorting, the code creates a new array `sorted_coordinates` by rearranging the coordinates based on the sorted order, which is $O(n)$.

   **Time Complexity for Sorting**: $O(t \times n \log n)$.

#### 4. **Computing the Smallest Path**
   - The `compute_smallest_path` function iterates through each of the sorted coordinates in $O(n)$ time.
   - For each coordinate, it checks if the target coordinate can be reached by only moving "up" (U) or "right" (R). This involves a constant-time check and, if infeasible, it returns immediately.
   - If the target can be reached, the code calculates the distance along $x$ and $y$ axes.
   - For each coordinate, it adds the steps to reach it. Moving each step and updating coordinates takes $O(1)$, and each package requires a bounded number of steps (no more than $x + y$ operations per package).

   Overall, this is an $O(n)$ operation since it iterates through all coordinates once and performs a bounded amount of work for each coordinate.

   **Time Complexity for Smallest Path Calculation**: $O(t \times n)$.

#### 5. **Printing Results**
   - Printing “YES” or “NO” and the path string is $O(n)$ for each test case.

   **Time Complexity for Printing**: $O(t \times n)$.

#### **Total Time Complexity**

Summing up the complexities for each step, the dominant term is $O(t \times n \log n)$ from the sorting step. Thus, the overall time complexity is:

$$O(t \times n \log n)$$


where:
- $t$ is the number of test cases,
- $n$ is the number of packages per test case.

This complexity indicates that the code's performance primarily depends on the sorting operation within each test case.


 - I dont fully agree with the response give by *ChatGPT*. It has broken in small parts the calculus of time complexity like we have done, but it wasnt accurate in my opinion on the study of the algorithm `compute_smallest_path` because it has made the assumption that the size of the path $p$ is $O(1)$. 

- But if $p$ is $O(n)$ for example the **Total time complexity** is not $O(t \times \log(n))$ but it would be $O(t \times n^2)$ which would be the worst case because the function `compute_smallest_path` would consume the most amount of time.

5. Assume now that the robot can also move towards the left or downwards, and consider the greedy approach: from the current location go to the closest package. Notice that now we can always collect all packages. Prove that the greedy algorithm is optimal (i.e., it minimizes the total distance traveled), or provide a counterexample showing that it is not.   

The proof of optimality for this greedy approach relies on the optimal substructure of the problem. Specifically, we show that if the greedy choice is not optimal at some step, swapping the greedy move with an alternative will equalize or increase the total distance traveled. 

#### **Step-by-Step Analysis**: 

The algorithm is based on the calculus of the **Manhattan Distance** that minimizes by definition the number of single steps between two points.

**Initial State**: The robot starts at some position $(𝑥_0,𝑦_0)$ and is tasked with collecting a set of packages, each located at $( 𝑥_𝑖 , 𝑦_𝑖 )$.

**Greedy Step**: At each step, the robot moves to the closest package (because we have previously sorted the list of packages). 
Let’s assume the greedy algorithm at some stage goes from the current position $(x_k,y_k)$ to some package $( 𝑥_𝑞 , 𝑦_𝑞)$, and then from $(x_q ​ ,y_q ​ )$ to $(x_p ​ ,y_p ​ )$ selecting the closest package.

 Now, let’s consider an another algorithm alternative (not *greedy*) swapping the two steps: first move from $(x_k ​ ,y_k ​ )$ to $(x_p ​ ,y_p ​ )$, and then move from $( 𝑥_𝑝 , 𝑦_𝑝 )$ to $(x_q ​ ,y_q ​)$. 

 The total distance (equal to the **Manhattan Distance**) for the greedy choice is: $𝐷_{greedy} = |x_k − 𝑥_𝑝|+ |𝑦_𝑘 − 𝑦_𝑝| + |𝑥_𝑝 − 𝑥_𝑞| + |𝑦_𝑝 − 𝑦_𝑞|$

The total distance for the alternative choice is: $D_{alternative} ​ = ∣x_k ​− x_q ∣ + ∣y_k− y_q∣ + ∣x_q − x_p∣ + |y_q​ − y_p|$

The key observation here is that, by the triangle **inequality for Manhattan distance**, we have: 
$$ |x_k ​ −x_p ∣ + ∣y_k ​− y_p∣ + |𝑥_𝑝 − 𝑥_𝑞| + |𝑦_𝑝 − 𝑦_𝑞| \le ∣ x_k − x_q∣ + ∣y_k​ − y_q∣ + ∣x_q − x_p∣ + ∣y_q​ − y_p∣$$
Hence, moving directly from  $(x_k,y_k )$ to $(x_p,y_p)$ and then to $(x_q ,y_q)$ will not result in a greater distance than the alternative move. 
 
Therefore, the greedy approach (choosing the closest package at each step) is at least as good as any alternative strategy in terms of total distance.

For completeness, we have implemented also the function that simulates this extended case.

In [ ]:
import numpy as np 
import functions

with open("algorithmic_input.txt") as file:

    # Select the number of Test Cases
    t = int(file.readline().strip())
    # Iterate over the number fo test cases
    for _ in range(t):
        
        # Extract the number of Packages to collect
        number_packages = int(file.readline().strip())

        # Store all the input coordinate into the Numpy Array
        list_of_coordinates = np.array([tuple(map(int, file.readline().strip().split())) for _ in range(number_packages)] , dtype = int)

        # I compute the distance from the (0,0) point to sort the list
        distances = np.sqrt(list_of_coordinates[:, 0]**2 + list_of_coordinates[:, 1]**2)

        # I have used the argsort function in order to return the indices of the distances ordered in ascendent way
        sorted_indices = np.argsort(distances)

        # I return the list ordered
        sorted_coordinates = list_of_coordinates[sorted_indices]

        # Execute the function to return the smallest path given an array of sorted coordinates
        functions.extended_compute_smallest_path(sorted_coordinates)

    




YES
RUUURRRRUU
YES
RLU
YES
RRRRUUU


We can see that in this case we are able to collect all the packages, so "NO" is not printed in the output